In [2]:
#needed imports from Pandas, Numpy, Pickle, Matplot, Keras
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pprint
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

if os.path.exists("best_model.pkl"): os.remove("best_model.pkl")

Using TensorFlow backend.


In [50]:
#loading raw data
path = 'MovementAAL/new_dataset/stream_0000'
sequences = list()

for i in range (1,10):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["delta1", "delta2", "delta3", "delta4", "delta5", "delta6", "delta7", "delta8", "delta9"])
    values = df.values
    sequences.append(values)
    
print(len(sequences))

path = 'MovementAAL/new_dataset/stream_000'

for i in range(10, 100):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["delta1", "delta2", "delta3", "delta4", "delta5", "delta6", "delta7", "delta8", "delta9"])
    sequences.append(values)
    
print(len(sequences))

path = 'MovementAAL/new_dataset/stream_00'

for i in range(100, 201):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["delta1", "delta2", "delta3", "delta4", "delta5", "delta6", "delta7", "delta8", "delta9"])
    sequences.append(values)

print(len(sequences))

9
99
200


In [51]:
#target values
targets = pd.read_csv('MovementAAL/new_dataset/MovementAAL_target.csv')
targets = targets.values[:200,1]
print(len(targets))
print(targets)

200
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [52]:
#Loading grouping
groups = pd.read_csv('MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:200,1]
print(len(groups))
print(groups)

200
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [53]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
print(pd.Series(len_sequences).describe())

count    200.0
mean      55.0
std        0.0
min       55.0
25%       55.0
50%       55.0
75%       55.0
max       55.0
dtype: float64


In [54]:
#Padding the sequence with the values in last row to max length
to_pad = 55
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   

    to_concat = np.repeat(one_seq[-1], n).reshape(9, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)


In [55]:
#truncate the sequence to length 60
seq_len = 55
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')
print(final_seq)

[[[-1. -1. -2. ... -2. -1. -2.]
  [ 0. -1. -2. ... -2. -1. -1.]
  [-1. -1. -2. ... -2.  0. -1.]
  ...
  [ 0.  0. -2. ... -2. -2. -1.]
  [ 0.  0. -3. ... -2. -2.  0.]
  [ 0. -1. -3. ... -3. -2.  0.]]

 [[ 0.  0. -2. ... -4. -3.  0.]
  [ 0.  0. -1. ... -1. -2. -1.]
  [ 0.  0.  0. ... -1. -1. -1.]
  ...
  [-1.  0.  0. ... -4. -1. -1.]
  [-1. -1.  0. ... -4. -1. -2.]
  [-1. -1.  0. ... -3. -1. -2.]]

 [[ 0.  1.  0. ... -1. -2. -2.]
  [ 0.  1. -1. ... -1.  0. -2.]
  [ 0.  0. -1. ... -1.  0. -2.]
  ...
  [ 1.  1. -1. ... -3. -1. -1.]
  [ 1.  0. -1. ... -3. -2.  0.]
  [ 0.  0. -1. ... -3. -1.  0.]]

 ...

 [[ 1.  1. -1. ... -1.  0.  1.]
  [ 1.  1.  0. ...  0.  0.  2.]
  [ 0. -2.  0. ...  1.  0.  2.]
  ...
  [ 1. -1.  0. ...  2. -1.  0.]
  [ 1. -1. -1. ...  1.  0.  1.]
  [ 1.  0. -1. ...  1.  0.  1.]]

 [[ 1.  1. -1. ... -1.  0.  1.]
  [ 1.  1.  0. ...  0.  0.  2.]
  [ 0. -2.  0. ...  1.  0.  2.]
  ...
  [ 1. -1.  0. ...  2. -1.  0.]
  [ 1. -1. -1. ...  1.  0.  1.]
  [ 1.  0. -1. ...  1.  0.  

In [56]:
print(len(groups))
print(len(final_seq))
#Training data based on group 2
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
#validation data based on group 1
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
#test data based on group 3
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

#train target based on group 2
train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
#validation target based on group 1
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
#test target based on group 3
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

print('train {} validation {} test {} \n target: train {} validate {} test {}'.format(len(train), len(validation), len(test), len(train_target), len(validation_target), len(test_target)))

200
200
train 66 validation 66 test 68 
 target: train 66 validate 66 test 68


In [57]:
#creating np.arrays for each dataset
train = np.array(train)
print(train)

validation = np.array(validation)
test = np.array(test)

#training target data
train_target = np.array(train_target)
train_target = (train_target+1)/2

print(train_target)

#validation target
validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

print(validation_target)

#test data and target test
test_target = np.array(test_target)
test_target = (test_target+1)/2

print(test_target)

[[[ 1.  1. -1. ... -1.  0.  1.]
  [ 1.  1.  0. ...  0.  0.  2.]
  [ 0. -2.  0. ...  1.  0.  2.]
  ...
  [ 1. -1.  0. ...  2. -1.  0.]
  [ 1. -1. -1. ...  1.  0.  1.]
  [ 1.  0. -1. ...  1.  0.  1.]]

 [[ 1.  1. -1. ... -1.  0.  1.]
  [ 1.  1.  0. ...  0.  0.  2.]
  [ 0. -2.  0. ...  1.  0.  2.]
  ...
  [ 1. -1.  0. ...  2. -1.  0.]
  [ 1. -1. -1. ...  1.  0.  1.]
  [ 1.  0. -1. ...  1.  0.  1.]]

 [[ 1.  1. -1. ... -1.  0.  1.]
  [ 1.  1.  0. ...  0.  0.  2.]
  [ 0. -2.  0. ...  1.  0.  2.]
  ...
  [ 1. -1.  0. ...  2. -1.  0.]
  [ 1. -1. -1. ...  1.  0.  1.]
  [ 1.  0. -1. ...  1.  0.  1.]]

 ...

 [[ 1.  1. -1. ... -1.  0.  1.]
  [ 1.  1.  0. ...  0.  0.  2.]
  [ 0. -2.  0. ...  1.  0.  2.]
  ...
  [ 1. -1.  0. ...  2. -1.  0.]
  [ 1. -1. -1. ...  1.  0.  1.]
  [ 1.  0. -1. ...  1.  0.  1.]]

 [[ 1.  1. -1. ... -1.  0.  1.]
  [ 1.  1.  0. ...  0.  0.  2.]
  [ 0. -2.  0. ...  1.  0.  2.]
  ...
  [ 1. -1.  0. ...  2. -1.  0.]
  [ 1. -1. -1. ...  1.  0.  1.]
  [ 1.  0. -1. ...  1.  0.  

In [58]:
#adding the LSTM to the model and printing the summary
model = Sequential()
model.add(LSTM(200, input_shape=(seq_len, 9)))
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 200)               168000    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
Total params: 168,000
Trainable params: 168,000
Non-trainable params: 0
_________________________________________________________________


In [59]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=20, batch_size=132, callbacks=[chk], validation_data=(validation,validation_target))


Train on 66 samples, validate on 66 samples
Epoch 1/20
66/66 [==============================] - 3s 42ms/step - loss: 0.5147 - accuracy: 0.5000 - val_loss: 0.2583 - val_accuracy: 0.3788

Epoch 00001: val_accuracy improved from -inf to 0.37879, saving model to best_model.pkl
Epoch 2/20
66/66 [==============================] - 0s 7ms/step - loss: 0.1821 - accuracy: 0.5000 - val_loss: -0.0184 - val_accuracy: 0.3788

Epoch 00002: val_accuracy did not improve from 0.37879
Epoch 3/20
66/66 [==============================] - 0s 7ms/step - loss: -0.1178 - accuracy: 0.5000 - val_loss: -0.2977 - val_accuracy: 0.4091

Epoch 00003: val_accuracy improved from 0.37879 to 0.40909, saving model to best_model.pkl
Epoch 4/20
66/66 [==============================] - 0s 7ms/step - loss: -0.4147 - accuracy: 0.5000 - val_loss: -0.6088 - val_accuracy: 0.4545

Epoch 00004: val_accuracy improved from 0.40909 to 0.45455, saving model to best_model.pkl
Epoch 5/20
66/66 [==============================] - 0s 7ms/st

In [61]:
#loading the exported pkl model and testing the accuracy score
model = load_model('saved_model.pb')
from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
print('Length of test cases is {}'.format(len(test)))
print(test)
print('length of test predictions is {}'.format(len(test_preds)))
print(test_preds)
print('length of test_target is {}'.format(len(test_target)))
print(test_target)
accuracy_score(test_target, test_preds)
print(accuracy_score(test_target, test_preds))

OSError: Unable to open file (file signature not found)

In [87]:
import os

import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.load_model('/Users/Kelly/Desktop/Test/project_28/python_scripts/nine-sensor-model/lstm')

TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f833e280f10>

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.load_model('/Users/Kelly/Desktop/Test/project_28/python_scripts/nine-sensor-model/lstm')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               44000     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 404       
Total params: 54,504
Trainable params: 54,504
Non-trainable params: 0
_________________________________________________________________
